In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import torch
from pytorch_lightning import seed_everything

from utils.data import get_hsm_dataset, split_data, log_returns, get_solar_energy_dataset
from utils.metrics import MAPE, WAPE, MAE

from fourier_flows.SequentialFlows import FourierFlow, RealNVP, TimeFlow

In [2]:
hsm_dataset_path = "data/huge_stock_market_dataset/"
solar_energy_dataset_path = "data/solar_energy/"
models_dir = "models/"

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

val_size = 0.0
test_size = 0.0

T = 127
n_samples = 800 * 127  # number of samples generated by QuantGAN

cuda:0


# Fourier Flow

In [8]:
ts_iterator = get_hsm_dataset(hsm_dataset_path, selected_files=f"{hsm_dataset_path}/selected100.csv")
synthetic_path = f"{hsm_dataset_path}synthetic/FourierFlow/"

for ts_index, time_series in enumerate(ts_iterator):
    print(f"Time Series #{ts_index}")
    
    train_ts = log_returns(time_series)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]

    seed_everything(0)
    FF_model = FourierFlow(hidden=200, fft_size=len(train_ts), n_flows=10, normalize=False)

    FF_losses = FF_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = FF_model.sample(n_samples // len(train_ts))
    np.save(synthetic_path + f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, FF_model, FF_losses

Global seed set to 0


Time Series #0
step: 0 	/ 50 	-	loss: 1435.699


Global seed set to 0


step: 49 	/ 50 	|	loss: -11635.513
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 917.960


Global seed set to 0


...

step: 49 	/ 50 	|	loss: -9063.104
Finished training!
Time Series #97
step: 0 	/ 50 	-	loss: 3393.610
step: 49 	/ 50 	|	loss: -29642.854
Finished training!


Global seed set to 0


Time Series #98
step: 0 	/ 50 	-	loss: 3481.111
step: 49 	/ 50 	|	loss: -26574.676
Finished training!


Global seed set to 0


Time Series #99
step: 0 	/ 50 	-	loss: 774.763
step: 49 	/ 50 	|	loss: -5006.361
Finished training!


Time: 11 min 35 sec

In [13]:
ts_iterator = get_solar_energy_dataset(solar_energy_dataset_path, max_results=10)
synthetic_path = f"{solar_energy_dataset_path}synthetic/FourierFlow/"

for ts_index, time_series in enumerate(ts_iterator):
    print(f"Time Series #{ts_index}")
    
    train_ts = time_series[: 10_000]
    train_ts = log_returns(train_ts + 1e-9)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]

    seed_everything(0)
    FF_model = FourierFlow(hidden=200, fft_size=len(train_ts), n_flows=10, normalize=False)

    FF_losses = FF_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = FF_model.sample(n_samples // len(train_ts))
    np.save(synthetic_path + f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, FF_model, FF_losses

Global seed set to 0


Time Series #0
step: 0 	/ 50 	-	loss: 12285.785
step: 49 	/ 50 	|	loss: -53392.664
Finished training!


Global seed set to 0


Time Series #1
step: 0 	/ 50 	-	loss: 12285.741
step: 49 	/ 50 	|	loss: -54767.930
Finished training!


Global seed set to 0


...

Time Series #7
step: 0 	/ 50 	-	loss: 12287.970
step: 49 	/ 50 	|	loss: -54194.988
Finished training!


Global seed set to 0


Time Series #8
step: 0 	/ 50 	-	loss: 12286.185
step: 49 	/ 50 	|	loss: -55841.684
Finished training!


Global seed set to 0


Time Series #9
step: 0 	/ 50 	-	loss: 12287.316
step: 49 	/ 50 	|	loss: -51734.207
Finished training!


Time: 16 min 30 sec

# RealNVP

In [15]:
ts_iterator = get_hsm_dataset(hsm_dataset_path, selected_files=f"{hsm_dataset_path}/selected100.csv")
synthetic_path = f"{hsm_dataset_path}synthetic/RealNVP/"
seed_everything(0)

for ts_index, time_series in enumerate(ts_iterator):
    print(f"Time Series #{ts_index}")
    
    train_ts = log_returns(time_series)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]

    RealNVP_model = RealNVP(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

    RealNVP_losses = RealNVP_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = RealNVP_model.sample(n_samples // len(train_ts))
    np.save(synthetic_path + f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, RealNVP_model, RealNVP_losses

Global seed set to 0


Time Series #0
step: 0 	/ 50 	-	loss: 1471.588
step: 49 	/ 50 	|	loss: -8622.956
Finished training!
Time Series #1
step: 0 	/ 50 	-	loss: 975.932
step: 49 	/ 50 	|	loss: -4691.843
Finished training!
Time Series #2
step: 0 	/ 50 	-	loss: 980.449
step: 49 	/ 50 	|	loss: -4640.819
Finished training!
Time Series #3
step: 0 	/ 50 	-	loss: 1436.910
step: 49 	/ 50 	|	loss: -5758.834
Finished training!
Time Series #4
step: 0 	/ 50 	-	loss: 3636.094
step: 49 	/ 50 	|	loss: -18605.061
Finished training!
Time Series #5
step: 0 	/ 50 	-	loss: 434.827
step: 49 	/ 50 	|	loss: -2586.895
Finished training!
Time Series #6
step: 0 	/ 50 	-	loss: 1219.745
step: 49 	/ 50 	|	loss: -5989.792
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 695.225
step: 49 	/ 50 	|	loss: -4457.428
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 3252.448
step: 49 	/ 50 	|	loss: -16789.691
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 390.224
step: 49 	/ 50 	|	loss: -2010.325
Finished training!
Tim

Time: 8:22

In [4]:
ts_iterator = get_solar_energy_dataset(solar_energy_dataset_path, max_results=10)
synthetic_path = f"{solar_energy_dataset_path}synthetic/RealNVP/"
seed_everything(0)
start_point = 6
for _ in range(start_point): next(ts_iterator)

for ts_index, time_series in enumerate(ts_iterator, start_point):
    print(f"Time Series #{ts_index}")
    
    train_ts = time_series[: 10_000]
    train_ts = log_returns(train_ts + 1e-9)
    train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]

    RealNVP_model = RealNVP(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

    RealNVP_losses = RealNVP_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
                            learning_rate=1e-3, display_step=50)

    synth_data = RealNVP_model.sample(n_samples // len(train_ts))
    np.save(synthetic_path + f"selected{ts_index}.npy", synth_data)

    del train_ts, synth_data, RealNVP_model, RealNVP_losses

Global seed set to 0


Time Series #6
step: 0 	/ 50 	-	loss: 42233.957
step: 49 	/ 50 	|	loss: -98910.711
Finished training!
Time Series #7
step: 0 	/ 50 	-	loss: 49041.590
step: 49 	/ 50 	|	loss: -96680.203
Finished training!
Time Series #8
step: 0 	/ 50 	-	loss: 48661.266
step: 49 	/ 50 	|	loss: -95805.211
Finished training!
Time Series #9
step: 0 	/ 50 	-	loss: 44090.680
step: 49 	/ 50 	|	loss: -98405.523
Finished training!


Time: ~17 min

# TimeFlow

In [5]:
# synthetic_path = synthetic_path = f"{dataset_path}synthetic/TimeFlow/"
# ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")
# seed_everything(0)

# for ts_index, time_series in enumerate(ts_iterator):
#     print(f"Time Series #{ts_index}")
    
#     (train_ts, *_), *_ = split_data(time_series, val_size=val_size, test_size=test_size)
#     train_ts = log_returns(train_ts)
#     train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)]

#     TimeFlow_model = TimeFlow(hidden=200, T=len(train_ts), n_flows=10, normalize=False)

#     TimeFlow_losses = TimeFlow_model.fit(train_ts.values.reshape(1, - 1), epochs=50, batch_size=128, 
#                             learning_rate=1e-3, display_step=50)

#     synth_data = TimeFlow_model.sample(n_samples // len(train_ts))
#     np.save(synthetic_path + f"selected{ts_index}.npy", synth_data)

#     del train_ts, synth_data, TimeFlow_model, TimeFlow_losses

12 ts time: ~30 min

# Similarity

In [6]:
from tqdm import tqdm
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.special import kl_div

from utils.data import get_hsm_dataset, split_data, log_returns

In [8]:
results_dir = Path("results")

In [9]:
sj_div = lambda x, y: (kl_div(x, (x + y) / 2) + kl_div(y, (x + y) / 2)) / 2
min_max_norm = lambda x: (x - x.min()) / (x.max() - x.min())

In [14]:
start_dataset = 0
start_ts = 0

for dataset_path, dataset_name in ((Path("data/huge_stock_market_dataset/"), "hsm"),\
     (Path("data/solar_energy"), "se")):
    if dataset_name == "hsm" and start_dataset == 1: continue
    print(f"processing {dataset_name} dataset")
    for model in ("FourierFlow", "RealNVP"):
        synthetic_path = dataset_path / f"synthetic/{model}/"
        results = {"kl_div": [], "sj_div": []}
        if dataset_name == "hsm":
            ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected100.csv")
        else:
            ts_iterator = get_solar_energy_dataset(dataset_path, max_results=10)

        for ts_index, time_series in tqdm(enumerate(ts_iterator)):
            
            train_ts = log_returns(time_series if dataset_name == "hsm" else time_series[:10_000] + 1e-9)
            train_ts = train_ts[:(len(train_ts) // 4 * 4 + 1 if len(train_ts) % 4 > 0 else len(train_ts) - 3)].values.flatten()
            train_ts = min_max_norm(train_ts)
            
            synth_tss = np.load(synthetic_path / f"selected{ts_index}.npy")
            kl_div_res = sj_div_res = 0
            for synth_ts in synth_tss:
                synth_ts = min_max_norm(synth_ts)
                # synth_ts = np.histogram(synth_ts, bins=np.arange(start=0, stop=1, step=1/100))[0]
                # train_ts = np.histogram(train_ts, bins=np.arange(start=0, stop=1, step=1/100))[0]
                res = kl_div(synth_ts, train_ts)
                kl_div_res += np.where(np.isinf(res), 0, res).mean()
                sj_div_res += sj_div(synth_ts, train_ts).mean()
            results["kl_div"].append(kl_div_res / len(synth_tss))
            results["sj_div"].append(sj_div_res / len(synth_tss))
        
        pd.DataFrame(results).to_csv(results_dir / f"synth_{dataset_name}_sim_{model}.csv", index=False)

processing hsm dataset


100it [00:02, 47.59it/s]
100it [00:02, 47.42it/s]


processing se dataset


10it [00:00, 22.89it/s]
10it [00:00, 20.63it/s]
